In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import openai
import io
import pandas as pd
import tqdm
import sys
from datetime import datetime


sys.path.append('../hot_links')

from hot_links.vector_db_utils import *


2025-08-27 14:54:46.585 | INFO     | hot_links.config:<module>:11 - PROJ_ROOT path is: /workspaces/hot-links


## OpenAI Vector Store

In [4]:
now = datetime.now()
unix_time = now.timestamp()
# Make a time attribute


In [5]:
unix_time 

1756285588.642007

In [6]:
vector_store_id = create_vs("test_15")
file_id = push_file_to_cloud("test_15", "The key to the universe is 24")
res = add_file_to_db(file_id, vector_store_id, attributes={'time':unix_time})

In [7]:
response = client.responses.create(
    model="gpt-4.1",
    input="I need the key to the universe... What might it be? Please check in the vector store.",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
        "filters": {
            "type": "lt",
            "key": "time",
            "value": datetime.now().timestamp()
        }
    }]
)
print(response)
# Then query the store with filters

Response(id='resp_68aebf55f1d881908ae21ab8523f05de06121715ae4800a9', created_at=1756282710.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_68aebf56eff88190b6e9234825c1b68206121715ae4800a9', queries=['key to the universe'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_68aebf5801e48190be3e51b48fcfa36306121715ae4800a9', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-YAwezHhAVEmax1MrFbFr7E', filename='test_12.txt', index=29, type='file_citation')], text='The key to the universe is 24.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FileSearchTool(type='file_search', vector_store_ids=['vs_68aebf4484b48191981bd5f0c89a35ca'], filters=ComparisonFilter(key='time', type='lt', value=1756282708.7698

In [9]:
#response.error
response.output[1].content[0].text

'The key to the universe is 24.'

In [7]:
for vs in get_all_vs():
    print(vs.id,vs.name)

vs_68aecaa3def88191804f03510e690fb5 test_15
vs_68aec5911d54819189b3e643a653d59d test_14
vs_68aec56252388191924e885daeceed79 test_13
vs_68aebf4484b48191981bd5f0c89a35ca test_12
vs_68aebb80bbb08191ac68f228a08a83d4 test_11
vs_68aebae352f48191927c4b1260fb7f73 test_10
vs_68aeba2add0c81919da4e4d10fc47630 test_9
vs_68aeb9afea2081919f8be4f1ee611a82 test_8
vs_68aeb89b6c088191b75c037f1405d714 test_7
vs_68aeb734ba888191830f01f1ec997b47 test_6
vs_68aeb6ef7fb88191b1698338b66a6eec test_5
vs_68aeb6bfb31c81919e722bbd2f50c7b5 test_4
vs_68acb4af008c81919b4924472e209446 hot_links
vs_68acae441e188191a605f82b03a98305 hot_links
vs_68ac553757d88191873c6900f9c90c0c hot_links_final2
vs_68ac54fb91b0819180b65563cb88062e hot_links_final2
vs_68ac547571e881918c5b003281473561 hot_links_final
vs_68ac4cff5f1c81919b3bfda45ea803bb hot_links
vs_68ab6e552c348191b066ae9baa26b73f hot_links
vs_68ab680fb1908191b4c9cf36e59ab145 hot_links
vs_68ab595999388191ad48c47f6d09bae2 hot_links


In [8]:
delete_all_files()

16it [00:07,  2.11it/s]


## Try adding real files

In [3]:
df = pd.read_csv('../data/interim/snowballed.csv')

In [ ]:
vector_store_id = None

for vs in get_all_vs():
    print(vs.id,vs.name)
    
    if vs.name == 'hot_links':
        vector_store_id = vs.id
    
    # client.vector_stores.delete(vs.id)

vs_68aee23606108191acbb1b3f0dfc4f69 hot_links


In [ ]:
if not vector_store_id:
    vector_store_id = create_vs("hot_links")

In [ ]:
all_files = set(map(lambda x:x.filename, get_all_files()))
# Get all the files already in the vector store to skip omes already there

n_added = 0
n_skipped = 0

for row in tqdm.tqdm(df.head(8000).iterrows()):
    date,url,content = row[1][['webPublicationDate', 'webUrl','bodyContent']]

    if not url+'.txt' in all_files:
        
        logging.info("Adding %s", url)
        n_added+=1

        file_id = push_file_to_cloud(url, content)
        res = add_file_to_db(file_id, vector_store_id, attributes={'time':convert_date_to_epoch(date)})
    else:
        logging.info("Skipping %s", url)
        n_skipped+=1

logging.info("Finished processing files. Added: %d, Skipped: %d", n_added, n_skipped)
print("Finished processing files. Added: {:d}, Skipped: {:d}".format(n_added, n_skipped))

7350it [34:44,  1.26s/it]  

In [63]:
response = query_db(vector_store_id, "What were harry and meghan doing in 2022?", pd.to_datetime('2020-01-01').value//10**9)
print(response.output[1].content[0].text)

I couldn't find any information in the uploaded files specifically about what Harry and Meghan were doing in 2022. If you have another file or specific document section for me to check, please let me know! Otherwise, I can provide a general summary based on public knowledge.


In [64]:
response = query_db(vector_store_id, "What are the sanctions on vladimir putin?", pd.to_datetime('2023-01-01').value//10**9)
print(response.output[1].content[0].text)

Vladimir Putin, as President of Russia, has been personally targeted with specific sanctions by major Western powers in response to Russia’s invasion of Ukraine. Here’s a summary of the main sanctions against him as of mid-2022:

### Personal Sanctions

- **Travel Bans and Asset Freezes**: The US, European Union, UK, and other allied countries have imposed asset freezes and travel bans on Putin personally, making it illegal for people and businesses in those countries to provide him access to any assets or transact with him. This includes any known assets in these jurisdictions.
- **Sanctions on Putin’s Inner Circle**: Sanctions have also targeted Putin’s family members (such as alleged relatives and close associates), as well as several oligarchs considered to be in his inner circle. These measures aim to disrupt his support network and cut off financial flows to people seen as beneficiaries or sustainers of his regime.

### Economic and Symbolic Sanctions

- **Wider Economic Measures

In [62]:
response = query_db(vector_store_id, "What are the positive effects of breastfeeding for poor mothers?", pd.to_datetime('2020-01-01').value//10**9)
print(response.output[1].content[0].text)

Breastfeeding has several positive effects for poor mothers, particularly regarding the development and future success of their children:

1. **Better Cognitive Development for Children:** Studies show that children of poorer mothers who breastfeed do significantly better on tasks involving speaking, drawing, and comprehension. For example, breastfed children of disadvantaged mothers perform 8% better in cognitive ability tests up to age seven than those who are bottle-fed. At age three, these children score nearly 10 points higher in expressive language and around 8 points higher in school readiness than the average. These cognitive benefits persist at ages five and seven, including visuo-spatial skills and pictorial reasoning.

2. **Preparation for School:** The improved cognitive skills gained from breastfeeding prepare children better for starting primary school, which can be especially valuable for children from disadvantaged backgrounds.

3. **Cost Savings:** For poor mothers, br

In [67]:
results = search_db(vector_store_id, "What are the sanctions on Vladimir Putin?")

In [68]:
for r in results:
    print(r.score)
    print(r.filename)
    print(r.content)
    print()

0.8944001253027026
https://www.theguardian.com/world/2022/jun/29/vladimir-potanin-uk-sanctions-nickel-king-russia-putin.txt
[Content(text='Vladimir Potanin, known as the “Nickel King” and Russia’s second richest person, has become the latest Russian oligarch to be subjected to sanctions by the UK as ministers target “Putin’s inner circle”. The government on Wednesday announced sanctions against Potanin, who was deputy prime minister under Boris Yelstin; Anna Tsivileva, a cousin of Putin and president of the prominent Russian coal mining company JSC Kolmar Group; and Said Gutseriev, son of the already blacklisted Russian billionaire Mikhail Gutseriev. “As long as Putin continues his abhorrent assault on Ukraine, we will use sanctions to weaken the Russian war machine,” a government spokesperson said. “Today’s sanctions show that nothing and no one is off the table, including Putin’s inner circle.” The Foreign Office said Potanin was added to the list of more than 1,000 Russians hit by s

In [70]:
snippet = """Donald Trump imposed 50% tariffs on most US imports from India, making good on a threat to punish one of the world’s largest economies over its purchases of discounted Russian oil.

The tariffs, which came into effect just after midnight on Wednesday in Washington, risk inflicting significant damage on the Indian economy and further disrupting global supply chains.

US tariffs of 25% on Indian goods went into force earlier this month, but Trump announced plans to double the rate, citing New Delhi’s purchases of Russian oil, which the White House has argued is indirectly funding Russia’s war against Ukraine.

The US president has significantly increased US tariffs on goods from much of the world since reentering the White House in January, straining relations with economies of allies and rivals and fueling fears of higher inflation.

This latest move leaves Indian exporters facing among the highest US duties Trump has slapped on goods from overseas. Brazil is also grappling with 50% tariffs on its exports to the US."""

In [73]:
results = search_db(vector_store_id, "Which two articles would be most relevant to link to from an article\
    containing this snippet: \n{:s}".format(snippet))

In [74]:
for r in results:
    print(r.score)
    print(r.filename)
    print(r.content)
    print()

0.8160826278823857
https://www.theguardian.com/business/2022/jun/26/concerns-india-back-door-into-europe-for-russian-oil.txt
[Content(text='The stigma attached to the trade means some international companies involved in fuel supplies may attempt to mask its origins. Some energy firms rushed to cut shipments from Russia but industry watchers said some drivers in the south-east of England were still likely to still be filling up with diesel refined in Russia. State processors of oil are attempting to secure six-month supply contacts for Russian crude to India, Bloomberg reported this month. The trio of state refiners – Indian Oil Corp, Hindustan Petroleum and Bharat Petroleum – declined to answer questions on whether they were importing Russian oil or exporting it to Europe. Industry sources said tracking shipments of Russian oil to Europe via India is proving very difficult. “You’ll find that several shipments of crude will arrive at a port from different countries and be blended togeth

Based on [OpenAI documentation](https://platform.openai.com/docs/guides/tools-file-search#how-to-use)

In [22]:
from openai.types import FileObject

In [45]:
io.BytesIO('test'.encode('utf-8'))

In [14]:
make_file_object('dummy_3.txt', "Wine is made from grapes")

('dummy_3.txt.txt', <_io.BytesIO at 0x76e349562ca0>, 'text/markdown')

In [28]:
result.id

'file-RJNaMAB5ipLTiv696Tai1e'

In [6]:
def add_file_object(file_object):
    client.vector_stores.files.upload_and_poll(        # Upload file
    vector_store_id=vector_store.id,
    file=file_object   
)

In [7]:
client.vector_stores.files.upload_and_poll(        # Upload file
    vector_store_id=vector_store.id,
    file=make_file_object('dummy_3.txt', "Wine is made from grapes")   
)

VectorStoreFile(id='file-Vqdvu9xspCKE49JZ6WoYFr', created_at=1756148927, last_error=None, object='vector_store.file', status='completed', usage_bytes=1048, vector_store_id='vs_68acb4af008c81919b4924472e209446', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [8]:
client.files.retrieve("file-Vqdvu9xspCKE49JZ6WoYFr")

FileObject(id='file-Vqdvu9xspCKE49JZ6WoYFr', bytes=24, created_at=1756148925, filename='dummy_3.txt.txt', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None)

In [9]:
results = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="How is wine made?",
)

In [10]:
results

SyncPage[VectorStoreSearchResponse](data=[VectorStoreSearchResponse(attributes={}, content=[Content(text='Wine is made from grapes', type='text')], file_id='file-Vqdvu9xspCKE49JZ6WoYFr', filename='dummy_3.txt.txt', score=0.5208766983986627)], object='vector_store.search_results.page', search_query=['How is wine made?'], has_more=False, next_page=None)

In [11]:
add_file_object(make_file_object('xxx','lorem ipsum'))

In [ ]:
add_file_object(make_file_object('xxx',df.head()['bodyContent'].values[0]))

BadRequestError: Error code: 400 - {'error': {'message': 'File type not supported', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [85]:
df.head(10).apply(lambda row: make_file_object(row['webUrl'], row['bodyContent']), axis=1).apply(add_file_object)